In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/70-dog-breedsimage-data-set/dogs.csv
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/10.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/03.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/01.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/02.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/09.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/05.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/06.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/04.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/08.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/Cocker/07.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/American  Spaniel/10.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/American  Spaniel/03.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/American  Spaniel/01.jpg
/kaggle/input/70-dog-breedsimage-data-set/valid/American  Spaniel/02.jpg
/kaggle/input/70-dog-breedsimage-data

In [2]:
!pip install fastai

In [3]:
from fastcore.all import *
from fastai.vision.all import *

In [4]:
PATH = Path("../input/70-dog-breedsimage-data-set/train")
print(get_image_files(PATH))

[Path('../input/70-dog-breedsimage-data-set/train/Cocker/029.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/014.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/109.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/034.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/024.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/033.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/094.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/049.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/048.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/115.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/080.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/062.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/118.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/067.jpg'), Path('../input/70-dog-breedsimage-data-set/train/Cocker/106.j

In [5]:
def get_my_files(x):
    return get_image_files(str(PATH)+'/train') + get_image_files(str(PATH)+'/valid') 

In [6]:
#Custom Splitter - there are probably better/more efficient ways to write this, but hopefully this leads you in the right direction.
def FileSplitter():
    "Split `items` depending on the value of `mask`."
    def _func(pth_in):
        in_valid = False
        if 'valid' in str(pth_in):
            in_valid = True
        return in_valid
    def _inner(o, **kwargs): return FuncSplitter(_func)(o)
    return _inner
#splitter=FileSplitter(),

*I did not use the validation set from the dataset. I split the training set into training and validation set. Because I got some error like some label is not present in the test set but it is in the validation set. I was just looking for some quick results. I'll look into the issue and resolve it in the next version of this notebook. As soon as possible.*

In [7]:
dls = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=40),
    get_y = parent_label,
    item_tfms = [Resize(192, method="squish")]
).dataloaders(PATH, bs=32)

In [8]:
# dls2 = DataBlock(
#     blocks = (ImageBlock, CategoryBlock),
#     get_items = get_my_files,
#     splitter = FileSplitter(),
#     get_y = parent_label,
#     item_tfms = [Resize(192, method="squish")]
# ).dataloaders(str(PATH), bs=32)

In [9]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,1.553471,0.587639,0.169289,00:56


epoch,train_loss,valid_loss,error_rate,time
0,0.785906,0.504718,0.154814,00:34
1,0.432401,0.409232,0.124607,00:33
2,0.176323,0.387059,0.109503,00:34


In [10]:
path2 = '../input/70-dog-breedsimage-data-set/'
test_path = path2+'test'
import os

correct = 0
total = 0
for subdir, dirs, files in os.walk(test_path):
#     i=1
    for file in files:
#         print(os.path.join(subdir, file))
#         print(subdir)
#         print(str(subdir).split('/')[-1])
        img = PILImage.create(os.path.join(subdir, file))
#         print(str(test_path)+'/'+i+'/'+j)
        breed,_,probs = learn.predict(img)
        print(breed,  end=", ")
        # print(f"Probability it's a {breed}: {probs[0]:.4f}")
#         if(i==1):
#             PILImage.show(img)
#             i+=1
        if(breed == str(subdir).split('/')[-1]):
            correct+=1
        total+=1
    print()

Cocker, 

Cocker, 

Cocker, 

Cocker, 

Cocker, 

...

Corgi, 

Corgi, 

Corgi, 

Corgi, 

Corgi, 


In [11]:
accuracy = correct/total
print(accuracy)

0.9614285714285714


## Some sample tests

In [12]:
img = PILImage.create(test_path+'/Pit Bull/02.jpg')
breed,_,probs = learn.predict(img)
print(f"Breed: {breed},Probability: {max(probs)}",  end=", ")

Breed: Pit Bull,Probability: 0.9947099685668945, 

In [13]:
img = PILImage.create(test_path+'/Afghan/09.jpg')
breed,_,probs = learn.predict(img)
print(f"Breed: {breed},Probability: {max(probs)}",  end=", ")

Breed: Afghan,Probability: 0.9957017302513123, 

In [14]:
img = PILImage.create(test_path+'/Pug/05.jpg')
breed,_,probs = learn.predict(img)
print(f"Breed: {breed},Probability: {max(probs)}",  end=", ")

Breed: Pug,Probability: 0.9953011274337769, 

In [15]:
img = PILImage.create(test_path+'/Shih-Tzu/10.jpg')
breed,_,probs = learn.predict(img)
print(f"Breed: {breed},Probability: {max(probs)}",  end=", ")

Breed: Cockapoo,Probability: 0.5248610973358154, 

In [16]:
img = PILImage.create(test_path+'/Lhasa/02.jpg')
breed,_,probs = learn.predict(img)
print(f"Breed: {breed},Probability: {max(probs)}",  end=", ")

Breed: Lhasa,Probability: 0.9986963868141174, 

In [17]:
learn.export('dog-breed-classifier.pkl')